## 多元线性回归

#### 数据预处理

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
data=pd.read_csv(r'D:\Learning\ML_project\price.csv', encoding='gbk')

# 调整编码映射关系

# 使用 map 方法重新映射
data["Location"] = data["Location"].map({"郊区": 0,"次中心": 1,"市中心": 2})

# 用 label encoding 进行房屋地段特征预处理
data["Location"] = LabelEncoder().fit_transform(data["Location"])



In [3]:
from sklearn.preprocessing import OneHotEncoder

# 创建 OneHotEncoder 对象
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' 避免多重共线性

# 对 Developer 列进行编码
encoded_data = encoder.fit_transform(data[['Developer']])

# 将编码结果转换为 DataFrame
encoded_dfile = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Developer']))

# 将编码后的数据与原始数据合并
df_encoded = pd.concat([data.drop(columns=['Developer'],axis=1), encoded_dfile], axis=1) #axis=1是指列方向


In [4]:
import numpy as np
ys = data["Price"]   #标签
xs = df_encoded.drop(["Price"], axis=1)  #多元特征
ks=np.array([1, 5, 9, -8, 56, 78, 59])


In [6]:
# 将数据集划分为test和train

from sklearn.model_selection import train_test_split as t_t_s

x_train, x_test, y_train, y_test = t_t_s(xs, ys, random_state=0,test_size=0.2 )

x_train=x_train.to_numpy()
x_test=x_test.to_numpy()


In [5]:
# 计算多元线性回归的代价函数（均方误差）MSE

def cost_function(xs, ys, k, b):
    y_pred = (xs * k).sum(axis=1) + b  #用 xs @ k 也行
    cost = ((ys - y_pred)**2).mean()
    return cost

In [7]:
cost_function(x_train, y_train, ks, 1)

np.float64(25728.875)

In [138]:
#计算代价函数对 ks 和 b 的梯度。

import numpy as np

def count_grident(xs, ys, k, b):
    # b_grident = (k * xs + b - ys).mean()
    # k_grident = (xs * (k * xs + b - ys)).mean()
    # return k_grident, b_grident

    y_pred = (xs * k).sum(axis=1) + b  # 计算预测值
    b_gradient =   (y_pred - ys).mean() # 计算 b 的梯度
    # ks_gradient =  (xs.T @ (y_pred - ys)) / len(xs) # 计算 k 的梯度
    
    ks_gradient=np.zeros(xs.shape[1])

    for i in range(len(k)): 
        ks_gradient[i] =(xs[:, i] *(y_pred - ys)).mean()
    
    return ks_gradient, b_gradient

In [139]:
# k=np.array([1, 5, 9, -8, 56, 78, 59])
b=10
learn_rate=0.000001
print(cost_function(x_train, y_train, ks, b))
k_grident, b_grident=count_grident(x_train, y_train, ks, b)
for n in range(1000):
    ks=ks-k_grident*learn_rate
    b=b-b_grident*learn_rate
    k_grident, b_grident=count_grident(x_train, y_train, ks, b)

print(cost_function(x_train, y_train, ks, b), ks, b)

28597.625
1163.2942309709322 [-0.52035614  4.83120745  8.96121797 -8.01357992 55.99122751 77.99050044
 58.99142796] 9.982118848173288


In [148]:
def Final_grident(xs, ys, k_init, b_init, learn_rate, rounds):
    
    cost_history = []
    b_history = []
    k_history = [[]]

    k=k_init
    b=b_init
    
        # 更新参数并记录
    for n in range(rounds):
        
        k_grident, b_grident=count_grident(xs, ys, k, b)
        b=b-b_grident*learn_rate    
        k=k-k_grident*learn_rate
        

        k_history.append(k)
        b_history.append(b)

            # 计算当前代价并记录
        cost = cost_function(xs, ys, k, b)
        cost_history.append(cost)

        # 打印进度（可选）
        # if i % 100 == 0:
        #     print(f"Iteration {i}: Cost = {cost}")

    return k, b, cost_history, k_history, b_history

In [158]:

ks, b, cost_hist, k_hist, b_hist =Final_grident(x_train, y_train, ks, 10, 1e-3, 1000)

print(f"最佳的k为：{ks}, 最佳的b为:{b}, 最小代价为：{cost_function(x_train, y_train, ks, b)}")

d:\LCL\VScodeproject\.venv\lib\site-packages\numpy\_core\_methods.py:52: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\LCL\AppData\Local\Temp\ipykernel_20420\3096903021.py:15: RuntimeWarning: invalid value encountered in subtract
  k=k-k_grident*learn_rate
C:\Users\LCL\AppData\Local\Temp\ipykernel_20420\1718146408.py:4: RuntimeWarning: invalid value encountered in multiply
  y_pred = (xs * k).sum(axis=1) + b  #用 xs @ k 也行
C:\Users\LCL\AppData\Local\Temp\ipykernel_20420\146337593.py:10: RuntimeWarning: invalid value encountered in multiply
  y_pred = (xs * k).sum(axis=1) + b  # 计算预测值


最佳的k为：[nan nan nan nan nan nan nan], 最佳的b为:nan, 最小代价为：0.0


In [1]:
y_pred = (x_test * ks).sum(axis=1) +b
pd.DataFrame({"y_pred":y_pred, "y_test":y_test})

NameError: name 'x_test' is not defined

4851